<a href="https://colab.research.google.com/github/IamMarking/Tensorflow/blob/master/%E6%B7%B1%E5%85%A5_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TensorFlow 是一个非常强大的用来做**大规模数值计算**的库。其所擅长的任务之一就是实现以及训练深度神经网络。

在本教程中，我们将学到构建一个TensorFlow模型的基本步骤，并将通过这些步骤为MNIST构建一个深度卷积神经网络。

加载 MNIST 数据

这里，mnist 是一个轻量级的类。它以Numpy数组的形式存储着训练、校验和测试数据集。同时提供了一个函数，用于在迭代中获得minibatch，后面我们将会用到。

In [2]:
import tensorflow as tf

old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

tf.logging.set_verbosity(old_v)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz




这里，我们使用更加方便的InteractiveSession类。通过它，你可以更加灵活地构建你的代码。它能让你在运行图的时候，插入一些计算图，这些计算图是由某些操作(operations)构成的。这对于工作在交互式环境中的人们来说非常便利，比如使用IPython。如果你没有使用InteractiveSession，那么你需要在启动session之前构建整个计算图，然后启动该计算图。



In [0]:
sess = tf.InteractiveSession()

占位符，可以在TensorFlow运行某一计算时根据该占位符输入具体的值。

输出类别值y_也是一个2维张量，其中每一行为一个10维的one-hot向量,用于代表对应某一MNIST图片的类别。

In [0]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

一个变量代表着TensorFlow计算图中的一个值，能够在计算过程中使用，甚至进行修改

In [5]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Python代码的目的是用来构建这个可以在外部运行的计算图，以及安排计算图的哪一部分应该被运行

In [0]:
y = tf.nn.softmax(tf.matmul(x,W) + b)  # 预测值

In [0]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))  # 损失函数

In [0]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [0]:
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [14]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
correct_prediction

<tf.Tensor 'Equal:0' shape=(?,) dtype=bool>

In [15]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
accuracy

<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [16]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.905


## 构建一个多层卷积网络

### 权重初始化

In [0]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

TensorFlow在卷积和池化上有很强的灵活性。我们怎么处理边界？步长应该设多大？在这个实例里，我们会一直使用vanilla版本。我们的卷积使用1步长（stride size），0边距（padding size）的模板，保证输出和输入是同一个大小。我们的池化用简单传统的2x2大小的模板做max pooling。为了代码更简洁，我们把这部分抽象成一个函数。

In [0]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [0]:
%config ZMQInteractiveShell.ast_node_interactivity='all'

In [24]:
W_conv1 = weight_variable([5, 5, 1, 32])
W_conv1
b_conv1 = bias_variable([32])
b_conv1

<tf.Variable 'Variable_8:0' shape=(5, 5, 1, 32) dtype=float32_ref>

<tf.Variable 'Variable_9:0' shape=(32,) dtype=float32_ref>

In [0]:
x_image = tf.reshape(x, [-1,28,28,1])